In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from enum import Enum
from typing import Dict, List, Tuple


In [ ]:
class FattyAcidType(Enum):
    SATURATED = "saturated"
    MONOUNSATURATED = "monounsaturated"
    POLYUNSATURATED = "polyunsaturated"


In [ ]:
class FattyAcidMetabolism:
    def __init__(self, smiles: str):
        self.molecule = Chem.MolFromSmiles(smiles)
        if not self.molecule:
            raise ValueError("Invalid SMILES string")
        self.chain_length = self._get_carbon_chain_length()
        self.double_bonds = self._count_double_bonds()
        self.double_bond_positions = self._get_double_bond_positions()
        self.type = self._determine_fatty_acid_type()

    def _get_carbon_chain_length(self) -> int:
        return len([atom for atom in self.molecule.GetAtoms() if atom.GetSymbol() == 'C'])

    def _count_double_bonds(self) -> int:
        pattern = Chem.MolFromSmarts('C=C')
        return len(self.molecule.GetSubstructMatches(pattern))

    def _get_double_bond_positions(self) -> List[int]:
        pattern = Chem.MolFromSmarts('C=C')
        matches = self.molecule.GetSubstructMatches(pattern)
        return [match[0] for match in matches]

    def _determine_fatty_acid_type(self) -> FattyAcidType:
        if self.double_bonds == 0:
            return FattyAcidType.SATURATED
        elif self.double_bonds == 1:
            return FattyAcidType.MONOUNSATURATED
        else:
            return FattyAcidType.POLYUNSATURATED


In [ ]:
    def calculate_activation_energy(self) -> Dict[str, float]:
        return {
            'ATP_cost': 2,
            'description': 'Conversion to acyl-CoA via acyl-CoA synthetase'
        }

    def calculate_atp_yield(self) -> dict:
        activation_cost = self.calculate_activation_energy()['ATP_cost']
        cycles = (self.chain_length - 2) // 2

        fadh2_per_cycle = 1
        nadh_per_cycle = 1
        acetyl_coa_per_cycle = 1

        extra_nadh = self.double_bonds if self.type != FattyAcidType.SATURATED else 0

        is_odd_chain = self.chain_length % 2 != 0
        propionyl_coa_atp = 13 if is_odd_chain else 0

        total_fadh2 = cycles * fadh2_per_cycle
        total_nadh = (cycles * nadh_per_cycle) + extra_nadh
        total_acetyl_coa = cycles * acetyl_coa_per_cycle
        if is_odd_chain:
            total_acetyl_coa -= 1

        total_atp = (
            (total_fadh2 * 1.5) +
            (total_nadh * 2.5) +
            (total_acetyl_coa * 10) +
            propionyl_coa_atp -
            activation_cost
        )

        return {
            'activation_cost': activation_cost,
            'β-oxidation_cycles': cycles,
            'FADH2_ATP': total_fadh2 * 1.5,
            'NADH_ATP': total_nadh * 2.5,
            'acetyl_CoA_ATP': total_acetyl_coa * 10,
            'propionyl_CoA_ATP': propionyl_coa_atp,
            'is_odd_chain': is_odd_chain,
            'unsaturation_type': self.type.value,
            'double_bonds': self.double_bonds,
            'total_ATP': total_atp
        }


In [ ]:
    def visualize_steps(self):
        steps = []
        descriptions = []

        steps.append(self.molecule)
        descriptions.append("Initial fatty acid")

        activated_smiles = self._simulate_activation(Chem.MolToSmiles(self.molecule))
        activated_mol = Chem.MolFromSmiles(activated_smiles)
        steps.append(activated_mol)
        descriptions.append("Activated fatty acid (acyl-CoA)")

        current_mol = activated_mol
        cycle = 1

        while len([atom for atom in current_mol.GetAtoms() if atom.GetSymbol() == 'C']) > 2:
            if self.type != FattyAcidType.SATURATED and cycle in self._get_double_bond_positions():
                isomerized_smiles = self._simulate_isomerization(Chem.MolToSmiles(current_mol))
                isomerized_mol = Chem.MolFromSmiles(isomerized_smiles)
                steps.append(isomerized_mol)
                descriptions.append(f"Cycle {cycle}: Isomerization")
                current_mol = isomerized_mol

            oxidized_smiles = self._remove_two_carbons(Chem.MolToSmiles(current_mol))
            current_mol = Chem.MolFromSmiles(oxidized_smiles)
            steps.append(current_mol)
            descriptions.append(f"Cycle {cycle}: β-oxidation")
            cycle += 1

        fig, axes = plt.subplots(len(steps), 1, figsize=(10, 5*len(steps)))
        if len(steps) == 1:
            axes = [axes]

        for idx, (mol, desc) in enumerate(zip(steps, descriptions)):
            img = Draw.MolToImage(mol)
            axes[idx].imshow(img)
            axes[idx].axis('off')
            axes[idx].set_title(f'Step {idx+1}: {desc}')

        plt.tight_layout()
        return fig

    def _simulate_activation(self, smiles: str) -> str:
        return smiles.replace("O", "SCoA")

    def _simulate_isomerization(self, smiles: str) -> str:
        return smiles

    def _remove_two_carbons(self, smiles: str) -> str:
        mol = Chem.MolFromSmiles(smiles)
        atoms = mol.GetAtoms()
        n_carbons = len([atom for atom in atoms if atom.GetSymbol() == 'C'])
        return f"C{'C' * (n_carbons-4)}(=O)SCoA"


In [ ]:
# Example: Palmitic acid (C16:0)
smiles = "CCCCCCCCCCCCCCCC(=O)O"
fa = FattyAcidMetabolism(smiles)

# Display type and ATP yield
print("Fatty acid type:", fa.type)
print("ATP yield breakdown:")
for k, v in fa.calculate_atp_yield().items():
    print(f"  {k}: {v}")

# Visualize
fa.visualize_steps()
